# Setting up in the new network / for new accounts:
- This notebook allows to prepare wallets, tokens and contracts for running scenarios.
- You would need to update the consts in the [/scripts/defaults.py](../../scripts/defaults.py) to use deployed tokens and contracts in the scenarios.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `100.364256 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `90.674712709 ꜩ`


### Reveal tezos key if needed:

In [ ]:
opg = tezos_account.reveal().send()
tezos_account.wait(opg)

### Deploying XTZ ticketer:

In [ ]:
from scripts.helpers.contracts.xtz_ticketer import XtzTicketer

opg = XtzTicketer.originate(tezos_account).send()
tezos_account.wait(opg)
xtz_ticketer = XtzTicketer.from_opg(tezos_account, opg)
xtz_ticketer_address_hex = xtz_ticketer.address.encode().hex()

click.echo('- Ticketer address: ' + wrap(accent(xtz_ticketer.address)))
click.echo('- Ticketer address hex: ' + wrap(accent(xtz_ticketer_address_hex)))

### Deploying XTZ ticket helper:

In [ ]:
from pytezos import ContractInterface

helper = tezos_account.using(shell='ghostnet').contract('KT1VEjeQfDBSfpDH5WeBM5LukHPGM2htYEh3')

storage = {
    'exchanger': xtz_ticketer.address,
    'request_deposit': None
}

opg = ContractInterface.from_micheline(helper.script()['code']).using(
    shell=tezos_account.shell,
    key=tezos_account.key,
).originate(initial_storage=storage).send()

### Deposit XTZ to L2:
- This call will fund default etherlink_account on Etherlink side by utilizing native deposit bridge.

In [2]:
opg_hash = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    receiver_address=etherlink_account.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1MiHpNdyMmVegAwrsB4z6wV4kdK4YoFSEZ`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/quebecnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1AvSV5JHRGaHtxjb6xsfYENV8arBwDBq2p`
      * Receiver address: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
      * Amount (mutez): `90_000_000`
Successfully executed XTZ deposit, tx hash: `ooYgiN3ZaEGEKp9hF6wwk7cDjLmUFNxntzHGRdNyy7CAYT2eocc`


### Deploy L1 token:
- This call will deploy new token, that can be used as a testing token in the scenarios.
- This token should be stored in the Ticketer contract storage to be used in other test scenarios.

In [6]:
from scripts.tezos.deploy_token import deploy_token

token = deploy_token.callback(
    token_type='FA2',
    token_id=9000,
    total_supply=70_000_000,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
    silent=False,
)

Deploying mock token contract `!NO-TOKEN-SYMBOL`:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/quebecnet/`
  - Params:
Successfully deployed `!NO-TOKEN-SYMBOL` `FA2` Token, address: `KT1BP2ErUg8N6d9DVk2TfuDC8K4NXMrbh4vj`


- You will need to update the `TEZOS_TOKEN_ADDRESS` and `TEZOS_TOKEN_TYPE` variables in the [/scripts/defaults.py](../../scripts/defaults.py), to use this token contract in the test scenarios.

### Deploy bridge:
- This call will deploy bridge contracts on both Tezos and Etherlink side for the provided token.
- You can both use existing token from the network or deploy you own test token.

In [ ]:
from scripts.bridge_token import bridge_token

tezos_bridge_contracts = bridge_token.callback(
    token_address=token.address,
    token_type='FA2',
    token_id=9000,
    token_decimals=0,
    token_symbol='vSecT',
    token_name='Various Security Testing Scenarios',
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
    kernel_address=KERNEL_ADDRESS,
    skip_confirm=True,
)

ticketer = tezos_bridge_contracts['ticketer']
erc20 = tezos_bridge_contracts['erc20']
token_bridge_helper = tezos_bridge_contracts['token_bridge_helper']

- You will need to update the `TICKETER_ADDRESS`, `ERC20_PROXY_ADDRESS`, `TOKEN_BRIDGE_HELPER_ADDRESS` variables in the [/scripts/defaults.py](../../scripts/defaults.py), to use these contracts in other scenarios.

In [10]:
click.echo("TICKETER_ADDRESS = '" + accent(ticketer.address) + "'")
click.echo("ERC20_PROXY_ADDRESS = '" + accent(erc20.address) + "'")
click.echo("TOKEN_BRIDGE_HELPER_ADDRESS = '" + accent(token_bridge_helper.address) + "'")

TICKETER_ADDRESS = 'KT1BgbBBYh2eiNSNwjGzyiHmTrX1W8ptpDYm'
ERC20_PROXY_ADDRESS = '0x22D47960Bf1A741176ed29d2517d711d7Ef19eB4'
TOKEN_BRIDGE_HELPER_ADDRESS = 'KT1V4VLWWBaSzjHok7kMNStF3FWaMpMmwMwv'


### Deploy TicketRouterTester:
- This is the tester-contract allowing to make arbitrary calls to the rollup, ticketer, accept tickets with different entrypoints, redirect them, mint tickets.

In [15]:
from scripts.helpers.scenarios import deploy_ticket_router_tester

ticket_router_tester = deploy_ticket_router_tester(
    tezos_account,
    TEZOS_RPC_URL,
)

Deploying TicketRouterTester:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/quebecnet/`
Successfully deployed TicketRouterTester, address: `KT1TLcwLXsbqC1sPMvVHgrfREsbnFYsbtwgK`


- You will need to update the `TICKET_ROUTER_TESTER_ADDRESS` variable in the [/scripts/defaults.py](../../scripts/defaults.py), to use this contracts in other scenarios.

### Wrap tokens to tickets:
- To use tickets, that represent FA token, directly from tezos account, they should be converted without sending to the rollup.
- There is a helper to do this:

In [3]:
from scripts.defaults import TICKETER_ADDRESS

ticketer = Ticketer.from_address(tezos_account, TICKETER_ADDRESS)
ticketer.address

'KT1BgbBBYh2eiNSNwjGzyiHmTrX1W8ptpDYm'

In [4]:
opg_hash = wrap_tokens_to_tickets(tezos_account, ticketer, 1_000_000)

Wrapping FA2 token, address: `KT1BP2ErUg8N6d9DVk2TfuDC8K4NXMrbh4vj`, id: `9000` tokens to tickets:
  - Token holder: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer: `KT1BgbBBYh2eiNSNwjGzyiHmTrX1W8ptpDYm`
  - Amount: `1_000_000`
Successfully wrapped, tx hash: `oonAp2AKdsnsEZfLvMmwfwZN9KbVrJCK78XFjrZQTLTeQ5QWBs3`
